In [32]:
import numpy as np
import random as rd
import secrets as st
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import math

**simulation**

In [33]:
time_step = 1
def initialize() :
    global N, L, x_positions, y_positions, directions, order_parameters
    x_positions = [ [rd.uniform(0, L) for _ in range(N)] ]
    y_positions = [ [rd.uniform(0, L) for _ in range(N)] ]
    directions = [ [rd.uniform(0, 2*np.pi) for _ in range(N)] ]

    #order parameter
    v_xs = np.cos(directions[-1])
    v_ys = np.sin(directions[-1])
    order_parameter = ( np.mean(v_xs)**2 + np.mean(v_ys)**2 )**0.5
    order_parameters = [order_parameter]

def update() :
    global R, v, etta, new_xs, new_ys, new_dirs, order_parameter
    xs = x_positions[-1]
    ys = y_positions[-1]
    v_xs = v*np.cos(directions[-1])
    v_ys = v*np.sin(directions[-1])
    #positions update
    new_xs = [ (xs[n] + time_step*v_xs[n])%L for n in range(N) ]
    new_ys = [ (ys[n] + time_step*v_ys[n])%L for n in range(N) ]
    #direction update
    new_dirs = []
    for n in range(N) :
        n_neighbors_vx = []
        n_neighbors_vy = []
        for j in range(N) :
            dx = min( xs[n]-xs[j], L-(xs[n]-xs[j]) )
            dy = min( ys[n]-ys[j], L-(ys[n]-ys[j]) )
            if (dx**2 + dy**2)**0.5 < R :
                n_neighbors_vx.append(v_xs[j])
                n_neighbors_vy.append(v_ys[j])
        n_neighbors_mean_vx = np.mean(n_neighbors_vx)
        n_neighbors_mean_vy = np.mean(n_neighbors_vy)
        if not n_neighbors_mean_vx == 0 :
          new_dir = math.atan2( n_neighbors_mean_vy, n_neighbors_mean_vx ) + rd.uniform(-etta/2, etta/2)
        new_dirs.append(new_dir)
    #order parameter
    v_xs = np.cos(directions[-1])
    v_ys = np.sin(directions[-1])
    order_parameter = ( np.mean(v_xs)**2 + np.mean(v_ys)**2 )**0.5

def observe() :
  x_positions.append(new_xs)
  y_positions.append(new_ys)
  directions.append(new_dirs)
  order_parameters.append(order_parameter)

**running**

In [34]:
N = 200
L = 5
v = 0.3
R = 1
etta = 0.5

def run_simulation() :
  global T
  update()
  observe()
  print(T)
  T+=1

In [ ]:
T = 0
initialize()
for _ in range(10) :
  run_simulation()

while True :
  run_simulation()
  List = order_parameters[-10:]
  if ( max(List) - min(List) ) < 0.01 :
    break

for _ in range(10) :
  run_simulation()
  List = order_parameters[-10:]
  if ( max(List) - min(List) ) < 0.01 :
    break

plt.plot([t for t in range(len(order_parameters))], order_parameters)
plt.show()

# animation
fig, ax = plt.subplots()
ax.set_xlim(0, L)
ax.set_ylim(0, L)
quiver = ax.quiver(x_positions[0], y_positions[0],
    np.cos(directions[0]), np.sin(directions[0]),
    color='blue', scale=20, width=0.005)

def update0(frame):
    quiver.set_offsets(np.column_stack((x_positions[frame], y_positions[frame])))
    quiver.set_UVC(np.cos(directions[frame]), np.sin(directions[frame]))
    return quiver,

ani = FuncAnimation(fig, update0, frames=T, interval=300, blit=True)
ax.set_title(f"N={N}   L={L}   v={v}   R={R}   etta={etta}   T={T}")
ani.save("animation_Vicsek.mp4")